Link to the original blog post: https://jina.ai/news/implementing-a-chat-history-rag-with-jina-ai-and-milvus-lite

## Implementing a Chat History RAG with Jina AI and Milvus Lite

### Install the prerequisites

In [ ]:
!pip install -U pymilvus
!pip install -U "pymilvus[model]"
!pip install langchain
!pip install langchain-community

### Restart the kernel after installing the prerequisites

### Download the dataset

In [ ]:
import os

if not os.path.exists("chat_history.json"):
    !wget https://raw.githubusercontent.com/jina-ai/workshops/main/notebooks/embeddings/milvus/chat_history.json

### Add the [Jina API key](https://jina.ai/embeddings) and [HuggingFace token](https://huggingface.co/docs/hub/en/security-tokens) as environment variables

Make sure that your HuggingFace token is set to READ to access the HuggingFace Hub on which [Mixtral 7B Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) is hosted.

In [ ]:
import getpass

os.environ["JINAAI_API_KEY"] = getpass.getpass(prompt="Jina API Key: ")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass(prompt="HuggingFace Token: ")

### Create the Milvus Lite collection

In [ ]:
from pymilvus import MilvusClient, DataType

# Specify a local file name as uri parameter of MilvusClient to use Milvus Lite
client = MilvusClient("milvus_jina.db")

schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, description="The Primary Key", is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, description="The Embedding Vector", dim=768)

index_params = client.prepare_index_params()
index_params.add_index(field_name="embedding", metric_type="COSINE", index_type="AUTOINDEX")

client.create_collection(collection_name="milvus_jina", schema=schema, index_params=index_params)

### Parse the chat history data

In [5]:
import json

with open("chat_history.json", "r", encoding="utf-8") as file:
    chat_data = json.load(file)

messages = []
metadatas = []

for channel in chat_data:
  chat_history = channel["chat_history"]
  chat_topic = channel["topic"]
  chat_expert = channel["expert_user"]
  for message in chat_history:
    text = f"""{message["user"]}: {message["message"]}"""
    messages.append(text)
    meta = {
        "time_stamp": message["time_stamp"],
        "file_name": message["file_name"],
        "parent_message_nr": message["parent_message_nr"],
        "channel": chat_topic,
        "expert": True if message["user"] == chat_expert else False
    }
    metadatas.append(meta)

### Embed the data using Jina Embeddings v2

In [6]:
from pymilvus.model.dense import JinaEmbeddingFunction

jina_ef = JinaEmbeddingFunction("jina-embeddings-v2-base-en")

embeddings = jina_ef.encode_documents(messages)

### Index the data in the Milvus Lite collection

In [7]:
collection_data = [{
    "message": message,
    "embedding": embedding,
    "metadata": metadata
} for message, embedding, metadata in zip(messages, embeddings, metadatas)]

data = client.insert(
    collection_name="milvus_jina",
    data=collection_data
)

### Query the chat history

In [8]:
query = "Who knows the most about encryption protocols in my team?"

#### Retrieve relevant messages and rerank them using Jina Reranker v1

In [9]:
from pymilvus.model.reranker import JinaRerankFunction

query_vectors = jina_ef.encode_queries([query])

results = client.search(
    collection_name="milvus_jina",
    data=query_vectors,
    limit=5,
)

results = results[0]

ids = [results[i]["id"] for i in range(len(results))]

results = client.get(
    collection_name="milvus_jina",
    ids=ids,
    output_fields=["id", "message", "metadata"]
)

jina_rf = JinaRerankFunction("jina-reranker-v1-base-en")

documents = [results[i]["message"] for i in range(len(results))]
reranked_documents = jina_rf(query, documents)

reranked_messages = []
for reranked_document in reranked_documents:
  idx = reranked_document.index
  reranked_messages.append(results[idx])

#### Generate the response using Mixtral 7B Instruct

In [10]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1")

prompt = """<s>[INST] Context information is below.\\n
        It includes the five most relevant messages to the query, sorted based on their relevance to the query.\\n
        ---------------------\\n
        {context_str}\\\\n
        ---------------------\\n
        Given the context information and not prior knowledge,
        answer the query. Please be brief, concise, and complete.\\n
        If the context information does not contain an answer to the query,
        respond with \\"No information\\".\\n
        Query: {query_str}[/INST] </s>"""

prompt = PromptTemplate(template=prompt, input_variables=["query_str", "context_str"])

llm_chain = prompt | llm

answer = llm_chain.invoke({"query_str":query, "context_str":reranked_messages})

print(f"\n\nANSWER:\n\n{answer}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


ANSWER

Based on the context information, User5 seems to be the most knowledgeable about encryption protocols in your team. They have mentioned that the new protocols enhance data security significantly, especially for cloud deployments.
